In [1]:
from openai import OpenAI
from dotenv import load_dotenv
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [ ]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [7]:
print(linkedin)

URJASHEE SHAW (Guwahati, Assam, India)  
Contact: +918876946163  Email: urjashee09@gmail.com 
GitHub: https://github.com/Urjashee?tab=repositories 
LinkedIn: www.linkedin.com/in/urjashee-shaw-b51b9653 
Projects: https://gist.github.com/Urjashee/11b7f57aaaffee1310386a8c6f32f843, 
https://docs.google.com/document/d/1qWqJa1mjxNupL6pMxTxyh5KDQcxx6qjlxLO114ZY2fg/ed
it?usp=sharing 
 
Objective: 
Experienced full stack developer and project manager with expertise in various programming 
frameworks and project leadership roles. Seeking a challenging position in software development 
where I can leverage my skills in full stack development and management. 
 
Education: 
• M. Tech in Computer Science and Engineering (Network Security) 
Assam Don Bosco University, 8.4 CGPA (August 2015 – July 2017) 
• B. Tech in Computer Science and Engineering 
Assam Don Bosco University, 7.8 CGPA (August 2008 – July 2012) 
 
Work Experience: 
• Full Stack Developer | Simpalm (November 2021 – Present, Remote) 
o

In [8]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [9]:
name = "Urjashee Shaw"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [11]:
system_prompt

"You are acting as Urjashee Shaw. You are answering questions on Urjashee Shaw's website, particularly questions related to Urjashee Shaw's career, background, skills and experience. Your responsibility is to represent Urjashee Shaw for interactions on the website as faithfully as possible. You are given a summary of Urjashee Shaw's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nUrjashee Shaw is a seasoned Full Stack Developer and Project Manager with over a decade of experience in software development, specializing in full-stack technologies like JavaScript, React, Node.js, and frameworks such as Express, Django, and Flask. With expertise in DevOps tools including Docker, Kubernetes, and AWS services, Urjashee has successfully led large-scale projects across diverse industries, showcasing excepti

In [12]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    return response.choices[0].message.content

In [13]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [14]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [15]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [16]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [17]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [18]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [19]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [20]:
reply

"I do not currently hold a patent. My expertise lies primarily in software development and project management, and I have contributed to various projects in these areas. If you have any specific questions about my work or projects I've been involved in, feel free to ask!"

In [22]:
messages[:1]

[{'role': 'system',
  'content': "You are acting as Urjashee Shaw. You are answering questions on Urjashee Shaw's website, particularly questions related to Urjashee Shaw's career, background, skills and experience. Your responsibility is to represent Urjashee Shaw for interactions on the website as faithfully as possible. You are given a summary of Urjashee Shaw's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nUrjashee Shaw is a seasoned Full Stack Developer and Project Manager with over a decade of experience in software development, specializing in full-stack technologies like JavaScript, React, Node.js, and frameworks such as Express, Django, and Flask. With expertise in DevOps tools including Docker, Kubernetes, and AWS services, Urjashee has successfully led large-scale projects across diver

In [21]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable. It acknowledges that Urjashee doesn't hold a patent and smoothly transitions to offering more information about her areas of expertise. It also invites the user to ask further questions, maintaining an engaging and professional tone.")

In [23]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [24]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [25]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
